In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# pd.options.display.float_format = '{:.5f}'.format
# np.set_printoptions(precision=6, suppress=True)

# 1727, 191
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/1727-nonsquare-train-from-1918-nonsquare-spmm-over-3s.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/191-nonsquare-test-from-1918-nonsquare-spmm-over-3s.csv')

In [2]:
# # 일부 feature -> MAPE 

# # Train + Valid
# pure_X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
# y_train = train['bz_smsm']

# # Test
# pure_X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
# y_test = test['bz_smsm']

In [3]:
# 일부 feature -> MAPE 168

# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['bz_smsm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['bz_smsm']

In [4]:
from sklearn.metrics import r2_score

def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [5]:
# # 전체 term을 log화
# X_train = np.log(pure_X_train)
# # ld, rd, ld*rd term은 기존 term으로 사용
# X_train = X_train.replace(X_train['ld'].to_numpy(), pure_X_train['ld'].to_numpy())
# X_train = X_train.replace(X_train['rd'].to_numpy(), pure_X_train['rd'].to_numpy())
# X_train = X_train.replace(X_train['ld*rd'].to_numpy(), pure_X_train['ld*rd'].to_numpy())

# # 전체 term을 log화
# X_test = np.log(pure_X_test)
# # ld, rd, ld*rd term은 기존 term으로 사용
# X_test = X_test.replace(X_test['ld'].to_numpy(), pure_X_test['ld'].to_numpy())
# X_test = X_test.replace(X_test['rd'].to_numpy(), pure_X_test['rd'].to_numpy())
# X_test = X_test.replace(X_test['ld*rd'].to_numpy(), pure_X_test['ld*rd'].to_numpy())

In [6]:
from scipy.optimize import nnls

# NNLS(Non-Negative Least Squares)회귀모델 정의 및 훈련데이터로 학습
# (모든 가중치가 0 또는 양수값을 가진다.)
nnls_coef, nnls_intercept =nnls(X_train.to_numpy(), y_train.to_numpy(), maxiter=10000)

In [7]:
# NNLS를 통해 나온 각 weight 추출
C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
C2=np.array(pd.DataFrame(nnls_coef).iloc[1])
C3=np.array(pd.DataFrame(nnls_coef).iloc[2])
C4=np.array(pd.DataFrame(nnls_coef).iloc[3])
C5=np.array(pd.DataFrame(nnls_coef).iloc[4])
C6=np.array(pd.DataFrame(nnls_coef).iloc[5])
C7=np.array(pd.DataFrame(nnls_coef).iloc[6])
C8=np.array(pd.DataFrame(nnls_coef).iloc[7])
C9=np.array(pd.DataFrame(nnls_coef).iloc[8])
C10=np.array(pd.DataFrame(nnls_coef).iloc[9])
C11=np.array(pd.DataFrame(nnls_coef).iloc[10])
C12=np.array(pd.DataFrame(nnls_coef).iloc[11])
C13=np.array(pd.DataFrame(nnls_coef).iloc[12])
C14=np.array(pd.DataFrame(nnls_coef).iloc[13])
C15=np.array(pd.DataFrame(nnls_coef).iloc[14])

# 테스트 데이터 예측값 도출
y_pred = C1*X_test['lr']+C2*X_test['lc']+C3*X_test['rc']+C4*X_test['ld']+C5*X_test['rd']+C6*X_test['lnnz']+C7*X_test['rnnz']+C8*X_test['lr*lc']+C9*X_test['lc*rc']+C10*X_test['lr*rc']+C11*X_test['lr*lc*rc']+C12*X_test['ld*rd']+C13*X_test['lr*rc*ld*rd']+C14*X_test['lr*lc*rc*ld*rd']+C15*X_test['lnnz*rnnz']

In [8]:
mape_list = {}
# 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
for idx,value in enumerate(y_test):
    mape_temp = {}
    predicate = int(y_pred[idx])
    mape = abs((value - predicate) / value) * 100
    mape_temp['pred'] = predicate
    mape_temp['real'] = value
    mape_temp['mape'] = mape
    mape_list[idx] = mape_temp
mape_list_sort = sorted(mape_list.values(), key=lambda x:(x['mape']), reverse=True)
mape_list_sort  

[{'pred': 87976, 'real': 3095, 'mape': 2742.520193861066},
 {'pred': 55077, 'real': 3236, 'mape': 1602.008652657602},
 {'pred': 67292, 'real': 4176, 'mape': 1511.3984674329504},
 {'pred': 52738, 'real': 3470, 'mape': 1419.8270893371757},
 {'pred': 50410, 'real': 3651, 'mape': 1280.7176116132566},
 {'pred': 73429, 'real': 5460, 'mape': 1244.8534798534797},
 {'pred': 51722, 'real': 3922, 'mape': 1218.765935747068},
 {'pred': 32907, 'real': 3014, 'mape': 991.8049104180492},
 {'pred': 44826, 'real': 4822, 'mape': 829.6142679386147},
 {'pred': 46631, 'real': 5127, 'mape': 809.5182367856447},
 {'pred': 58413, 'real': 7171, 'mape': 714.5725840189652},
 {'pred': 36982, 'real': 4691, 'mape': 688.3606906842891},
 {'pred': 37906, 'real': 4839, 'mape': 683.3436660467038},
 {'pred': 31105, 'real': 4131, 'mape': 652.9653836843379},
 {'pred': 34150, 'real': 4693, 'mape': 627.6795226933731},
 {'pred': 28897, 'real': 4584, 'mape': 530.3883071553229},
 {'pred': 35021, 'real': 5719, 'mape': 512.362301101

In [9]:
# 모델의 weight(가중치)와 bias(절편) 출력
print(nnls_coef)
print(nnls_intercept)

[1.21672907e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.16188458e-04 0.00000000e+00
 0.00000000e+00 3.08837530e-05 0.00000000e+00 0.00000000e+00
 6.27070417e-02 6.55179440e-07 1.39430967e-10]
2122587.6851577903


In [10]:
# nnls metric
print("R^2 : ", r2_score(y_test, y_pred))
print("\nMAPE : ", mean_absolute_percentage_error(y_test,y_pred))

R^2 :  0.8294545599504937

MAPE :  168.3022965601271
